In [1]:
import sys
import os

import warnings
warnings.filterwarnings("ignore")

sys.path.append(os.path.abspath(os.path.join("../scripts"))) 
import data_loader      

import pandas as pd

import dvc.api
import io

import datetime

from logger_class import App_Logger
# app_logger = App_Logger("../logs/dataLoading.log").get_app_logger()

In [2]:
# path = 'data/nb.csv'
# repo = './'
# version='nb_V0'

# data_url = dvc.api.get_url(
#     path=path,
# 	repo=repo,
# 	rev=version
# 	)

# tripdf = pd.read_csv(data_url)

tripdf = data_loader.load_data('../data/nb.csv')

In [3]:
# path = 'data/driver_locations_during_request.csv'
# repo = './'
# version='nb_V0'

# data_url = dvc.api.get_url(
#     path=path,
# 	repo=repo,
# 	rev=version
# 	)

# tripdf = pd.read_csv(data_url)

requestdf = data_loader.load_data('../data/driver_locations_during_request.csv')

In [4]:
def todatetime(df, column):
        df[column] = pd.to_datetime(df[column])
        return df

In [5]:
cols = ['Trip Start Time', 'Trip End Time']

for i in cols:
    todatetime(tripdf, i)

In [6]:
Trip_Org_lat = []
Trip_Org_long = []
Trip_Dest_lat = []
Trip_Dest_long = []

for i in range(len(tripdf)):
    Trip_Org_lat.append(float(tripdf['Trip Origin'][i].split(',')[0]))
    Trip_Org_long.append(float(tripdf['Trip Origin'][i].split(',')[1]))
    Trip_Dest_lat.append(float(tripdf['Trip Destination'][i].split(',')[0]))
    Trip_Dest_long.append(float(tripdf['Trip Destination'][i].split(',')[1]))

In [7]:
tripdf['Trip_Org_lat'] = Trip_Org_lat
tripdf['Trip_Org_long'] = Trip_Org_long
tripdf['Trip_Dest_lat'] = Trip_Dest_lat
tripdf['Trip_Dest_long'] = Trip_Dest_long

In [8]:
tripdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 536020 entries, 0 to 536019
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   Trip ID           536020 non-null  int64         
 1   Trip Origin       536020 non-null  object        
 2   Trip Destination  536020 non-null  object        
 3   Trip Start Time   534369 non-null  datetime64[ns]
 4   Trip End Time     536019 non-null  datetime64[ns]
 5   Trip_Org_lat      536020 non-null  float64       
 6   Trip_Org_long     536020 non-null  float64       
 7   Trip_Dest_lat     536020 non-null  float64       
 8   Trip_Dest_long    536020 non-null  float64       
dtypes: datetime64[ns](2), float64(4), int64(1), object(2)
memory usage: 36.8+ MB


In [9]:
requestdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1557740 entries, 0 to 1557739
Data columns (total 8 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   id             1557740 non-null  int64  
 1   order_id       1557740 non-null  int64  
 2   driver_id      1557740 non-null  int64  
 3   driver_action  1557740 non-null  object 
 4   lat            1557740 non-null  float64
 5   lng            1557740 non-null  float64
 6   created_at     0 non-null        float64
 7   updated_at     0 non-null        float64
dtypes: float64(4), int64(3), object(1)
memory usage: 95.1+ MB


In [10]:
requestdf.driver_action.value_counts()

rejected    1531837
accepted      25903
Name: driver_action, dtype: int64

In [11]:
tripdf_cop = tripdf.copy()
tripdf_cop.rename(columns={'Trip ID': 'order_id'}, inplace=True)

In [12]:
## Merge
completeddf = pd.merge(requestdf[requestdf['driver_action']=='accepted'], tripdf_cop, on='order_id', how='inner')

In [13]:
completeddf.to_csv('../data/completed_orders_descrip.csv', index=False)